In [4]:
# 导入所需的第三方库
import pandas as pd
import math
import numpy as np
import os
import collections
from functools import partial
import random
import time
import inspect
import importlib
from tqdm import tqdm
import pandas as pd
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import IterableDataset
from paddle.utils.download import get_path_from_url
from paddle.io import Dataset
# 导入paddlenlp所需的相关包
import paddlenlp as ppnlp
from paddlenlp.data import Pad, Stack, Tuple, Vocab
from paddlenlp.datasets import MapDataset
from paddle.dataset.common import md5file
from paddlenlp.datasets import DatasetBuilder

In [2]:
MODEL_NAME = "roberta-wwm-ext-large"
# 只需指定想要使用的模型名称和文本分类的类别数即可完成Fine-tune网络定义，通过在预训练模型后拼接上一个全连接网络（Full Connected）进行分类
model = ppnlp.transformers.RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=44) 
# 定义模型对应的tokenizer，tokenizer可以把原始输入文本转化成模型model可接受的输入数据格式。需注意tokenizer类要与选择的模型相对应，具体可以查看PaddleNLP相关文档
tokenizer = ppnlp.transformers.RobertaTokenizer.from_pretrained(MODEL_NAME)

[2022-03-22 12:02:23,493] [    INFO] - Already cached /Users/liruizhi/.paddlenlp/models/roberta-wwm-ext-large/roberta_chn_large.pdparams
/Users/liruizhi/opt/anaconda3/lib/python3.8/site-packages/paddle/fluid/dygraph/layers.py:1436: UserWarning: Skip loading for classifier.weight. classifier.weight is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/Users/liruizhi/opt/anaconda3/lib/python3.8/site-packages/paddle/fluid/dygraph/layers.py:1436: UserWarning: Skip loading for classifier.bias. classifier.bias is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
[2022-03-22 12:02:54,119] [    INFO] - Already cached /Users/liruizhi/.paddlenlp/models/roberta-wwm-ext-large/vocab.txt


In [5]:
class CHIPCTCDataSet(Dataset):
    def __init__(self, data_path, label_path):
        # 加载标签辞典
        self.label2id = self._load_label_dict(label_path)
        # 加载数据集
        self.data = self._load_data_from_source(data_path)
        # 加载标签列表
        self.label_list = list(self.label2id.keys())
        
    def _load_label_dict(self, label_path):
        with open(label_path, 'r', encoding='utf-8') as f:
            lines = [line.strip().split('\t', maxsplit=1)
                     for line in f.readlines()]
            lines = [(line[0], int(line[1])) for line in lines]
            label_dict = dict(lines)
        return label_dict
    
    def _load_data_from_source(self, data_path):
        data_set = []
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f.readlines():
                id, label, text = line.strip().split(',', maxsplit=2)
                example = {'text': text, 'label':self.label2id[label]}
                data_set.append(example)
        return data_set
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

In [14]:
# 加载训练和验证集
train_ds = CHIPCTCDataSet('/Users/liruizhi/Desktop/毕设数据处理/train_clean.csv', '/Users/liruizhi/Desktop/毕设数据处理/CHIP-CTC/CHIP-CTC_label_dict.txt')
label2id = train_ds.label2id
train_ds = MapDataset(train_ds)

In [16]:
# 定义数据加载和处理函数
def convert_example(example, tokenizer, max_seq_length=128, is_test=False):
    qtconcat = example["text"]
    encoded_inputs = tokenizer(text=qtconcat, max_seq_len=max_seq_length)  # tokenizer处理为模型可接受的格式 
    input_ids = encoded_inputs["input_ids"]
    token_type_ids = encoded_inputs["token_type_ids"]

    if not is_test:
        label = np.array([example["label"]], dtype="int64")
        return input_ids, token_type_ids, label
    else:
        return input_ids, token_type_ids

In [17]:
# 定义数据加载函数dataloader
def create_dataloader(dataset,
                      mode='train',
                      batch_size=1,
                      batchify_fn=None,
                      trans_fn=None):
    
    # 疑问点：trans_fn = convert_example
    if trans_fn:
        dataset = dataset.map(trans_fn)

    shuffle = True if mode == 'train' else False
    # 训练数据集随机打乱，测试数据集不打乱
    if mode == 'train':
        batch_sampler = paddle.io.DistributedBatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)
    else:
        batch_sampler = paddle.io.BatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)

    return paddle.io.DataLoader(
        dataset=dataset,
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True)

In [18]:
# 参数设置：
# 批处理大小，显存如若不足的话可以适当改小该值  
batch_size = 128
# 文本序列最大截断长度，需要根据文本具体长度进行确定，最长不超过512。 通过文本长度分析可以看出文本长度最大为48，故此处设置为48
max_seq_length = 128

In [19]:
# functools模块用于高阶函数：作用于或返回其他函数的函数。一般而言，任何可调用对象都可以作为本模块用途的函数来处理。  
# functools.partial返回的是一个可调用的partial对象，使用方法是partial(func,*args,**kw),func是必须要传入的，
# 而且至少需要一个args或是kw参数。

# plus3 = functools.partial(add, 3)
# plus5 = functools.partial(add, 5)
# plus3(4)
# 7
# plus3(7)
# 10
# plus5(10)
# 15

# 将数据处理成模型可读入的数据格式
trans_func = partial(
    convert_example,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length)

batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # token_type_ids
    Stack()  # labels
): [data for data in fn(samples)]

In [20]:
# 训练集迭代器
train_data_loader = create_dataloader(
    train_ds,
    mode='train',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func)

In [28]:
# 验证集迭代器
dev_data_loader = create_dataloader(
    dev_ds,
    mode='dev',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func)

In [21]:
# 定义超参，loss，优化器等
from paddlenlp.transformers import LinearDecayWithWarmup

# 定义训练配置参数：
# 定义训练过程中的最大学习率
learning_rate = 4e-5
# 训练轮次
epochs = 4
# 学习率预热比例
warmup_proportion = 0.1
# 权重衰减系数，类似模型正则项策略，避免模型过拟合
weight_decay = 0.01

num_training_steps = len(train_data_loader) * epochs
lr_scheduler = LinearDecayWithWarmup(learning_rate, num_training_steps, warmup_proportion)

# AdamW优化器
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ])

criterion = paddle.nn.loss.CrossEntropyLoss()  # 交叉熵损失函数
metric = paddle.metric.Accuracy()              # accuracy评价指标

In [22]:
# 定义模型训练验证评估函数
@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader):
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        correct = metric.compute(logits, labels)
        metric.update(correct)
        accu = metric.accumulate()
    print("eval loss: %.5f, accu: %.5f" % (np.mean(losses), accu))  # 输出验证集上评估效果
    model.train()
    metric.reset()
    return accu  # 返回准确率

In [23]:
# 固定随机种子便于结果的复现
seed = 1024
random.seed(seed)
np.random.seed(seed)
paddle.seed(seed)

In [24]:
# 模型训练：
import paddle.nn.functional as F

#save_dir = "checkpoint"
#if not  os.path.exists(save_dir):
    #os.makedirs(save_dir)

pre_accu=0
accu=0
global_step = 0
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, segment_ids, labels = batch
        logits = model(input_ids, segment_ids)
        loss = criterion(logits, labels)
        probs = F.softmax(logits, axis=1)
        correct = metric.compute(probs, labels)
        metric.update(correct)
        acc = metric.accumulate()

        global_step += 1
        if global_step % 10 == 0 :
            print("global step %d, epoch: %d, batch: %d, loss: %.5f, acc: %.5f" % (global_step, epoch, step, loss, acc))
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()
    # 每轮结束对验证集进行评估
#     accu = evaluate(model, criterion, metric, dev_data_loader)
#     print(accu)
#     if accu > pre_accu:
#         # 保存较上一轮效果更优的模型参数
#         save_param_path = os.path.join(save_dir, 'model_state.pdparams')  # 保存模型参数
#         paddle.save(model.state_dict(), save_param_path)
#         pre_accu=accu
    
# tokenizer.save_pretrained(save_dir)

KeyboardInterrupt: 